In [1]:
from controller import Supervisor
from math import pi

In [2]:
supervisor = Supervisor()

In [21]:
bottle = supervisor.getFromDef('bottle')

In [42]:
timestep = int(supervisor.getBasicTimeStep())

In [3]:
for i in range(supervisor.getNumberOfDevices()):
    print(supervisor.getDeviceByIndex(i).getName())

shoulder_pan_joint
shoulder_pan_joint_sensor
shoulder_lift_joint
shoulder_lift_joint_sensor
elbow_joint
elbow_joint_sensor
wrist_1_joint
wrist_1_joint_sensor
wrist_2_joint
wrist_2_joint_sensor
wrist_3_joint
wrist_3_joint_sensor
palm_finger_1_joint
palm_finger_1_joint_sensor
finger_1_joint_1
finger_1_joint_1_sensor
finger_1_joint_2
finger_1_joint_2_sensor
finger_1_joint_3
finger_1_joint_3_sensor
palm_finger_2_joint
palm_finger_2_joint_sensor
finger_2_joint_1
finger_2_joint_1_sensor
finger_2_joint_2
finger_2_joint_2_sensor
finger_2_joint_3
finger_2_joint_3_sensor
finger_middle_joint_1
finger_middle_joint_1_sensor
finger_middle_joint_2
finger_middle_joint_2_sensor
finger_middle_joint_3
finger_middle_joint_3_sensor


In [43]:
joints = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
finger_joints = ['finger_1_joint_1', 'finger_1_joint_2', 'finger_1_joint_3',
                 'finger_2_joint_1', 'finger_2_joint_2', 'finger_2_joint_3',
                 'finger_middle_joint_1', 'finger_middle_joint_2', 'finger_middle_joint_3']
joints = [supervisor.getDevice(joint) for joint in joints]
finger_joints = [supervisor.getDevice(joint) for joint in finger_joints]
finger_joint_limits = [[0.0595, 0.8], [0.01, 1], [-0.8, -0.0623],
                       [0.0595, 0.8], [0.01, 1], [-0.8, -0.0623],
                       [0.0595, 0.8], [0.01, 1], [-0.8,-0.0623]]

for dev in joints:
    dev.setPosition(float('inf'))
    dev.setVelocity(0)
    dev.setControlPID(1, 0, 0)
    dev.getPositionSensor().enable(timestep)

for dev in finger_joints:
    dev.getPositionSensor().enable(timestep)
    print(dev.getPositionSensor().getValue())


0.05951118786152998
0.010006672626415303
-0.7999962175677562
0.05950578182669263
0.009999949270415021
-0.7999962175677562
0.05950399885655573
0.010004053694486635
-0.7999962175676159


In [44]:
supervisor.step(timestep)

0

In [45]:
def get_joint_angles():
    angles = [joint.getPositionSensor().getValue() for joint in joints]
    angles[0] -= pi
    angles[1] += pi/2
    angles[3] += pi/2
    return np.array(angles)

In [46]:
def get_finger_angles():
    return np.array([joint.getPositionSensor().getValue() for joint in finger_joints])

In [ ]:
get_joint_angles()

In [47]:
import numpy as np

In [48]:
def move_to_config(target, duration=5):
    t0 = supervisor.getTime()
    v0 = np.zeros(6)
    vf = np.zeros(6)
    q0 = get_joint_angles()
    qf = np.array(target)
    a0 = np.zeros(6)
    af = np.zeros(6)
    tf = t0 + duration
    A = np.array([[1, t0, t0**2, t0**3, t0**4, t0**5],
                  [0, 1, 2*t0, 3*t0**2, 4*t0**3, 5*t0**4],
                  [0, 0, 2, 6*t0, 12*t0**2, 20*t0**3],
                  [1, tf, tf**2, tf**3, tf**4, tf**5],
                  [0, 1, 2*tf, 3*tf**2, 4*tf**3, 5*tf**4],
                  [0, 0, 2, 6*tf, 12*tf**2, 20*tf**3]])
    b = np.array([q0, v0, a0, qf, vf, af])
    x = [np.linalg.solve(A, b[:, i]) for i in range(6)]
    time0 = supervisor.getTime()
    iterations = 0
    end_effector_vel = []
    vel_jacob = [[], [], [], [], [], []]
    pos = [[], [], [], [], [], []]
    vel = [[], [], [], [], [], []]
    acc = [[], [], [], [], [], []]
    jerk = [[], [], [], [], [], []]
    time_arr = [[], [], [], [], [], []]
    while supervisor.getTime() <= tf:
        t = supervisor.getTime()
        for joint in joints:
            joint.setVelocity(x[joints.index(joint)][1] +
                              2*x[joints.index(joint)][2]*t +
                              3*x[joints.index(joint)][3]*t**2 +
                              4*x[joints.index(joint)][4]*t**3 +
                              5*x[joints.index(joint)][5]*t**4)


            idx = joints.index(joint)
            p = x[idx][0] + x[idx][1]*t + x[idx][2]*t**2 + x[idx][3]*t**3 + x[idx][4]*t**4 + x[idx][5]*t**5
            v = x[idx][1] + 2*x[idx][2]*t + 3*x[idx][3]*t**2 + 4*x[idx][4]*t**3 + 5*x[idx][5]*t**4
            a = 2*x[idx][2] + 6*x[idx][3]*t + 12*x[idx][4]*t**2 + 20*x[idx][5]*t**3
            j = 6*x[idx][3] + 24*x[idx][4]*t + 60*x[idx][5]*t**2
            time_arr[idx].append(t-time0)
            pos[idx].append(p)
            vel[idx].append(v)
            acc[idx].append(a)
            jerk[idx].append(j)
            v = x[idx][1] + 2*x[idx][2]*t + 3*x[idx][3]*t**2 + 4*x[idx][4]*t**3 + 5*x[idx][5]*t**4
            vel_jacob[idx].append(v)
        supervisor.step(timestep)
        iterations += 1
    for joint in joints:
        joint.setVelocity(0)
    timef = supervisor.getTime()
    error = np.abs(np.array(target) - get_joint_angles())*180/np.pi
    print('Iterações totais: ', iterations)
    print(timef-time0)
    print(np.max(error))
    print(np.mean(error))

In [49]:
def actuate_gripper(close=0):
    t0 = supervisor.getTime()
    v0 = np.zeros(9)
    vf = np.zeros(9)
    q0 = get_finger_angles()
    qf = np.hstack((np.array([lim[close] for lim in finger_joint_limits])))
    a0 = np.zeros(9)
    af = np.zeros(9)
    tf = t0 + 2
    A = np.array([[1, t0, t0**2, t0**3, t0**4, t0**5],
                  [0, 1, 2*t0, 3*t0**2, 4*t0**3, 5*t0**4],
                  [0, 0, 2, 6*t0, 12*t0**2, 20*t0**3],
                  [1, tf, tf**2, tf**3, tf**4, tf**5],
                  [0, 1, 2*tf, 3*tf**2, 4*tf**3, 5*tf**4],
                  [0, 0, 2, 6*tf, 12*tf**2, 20*tf**3]])
    b = np.array([q0, v0, a0, qf, vf, af])
    x = [np.linalg.solve(A, b[:, i]) for i in range(9)]
    time0 = supervisor.getTime()
    iterations = 0
    while supervisor.getTime() <= tf:
        t = supervisor.getTime()
        for idx, joint in enumerate(finger_joints):
            joint.setPosition(x[idx][0] + x[idx][1]*t + x[idx][2]*t**2 + x[idx][3]*t**3 + x[idx][4]*t**4 + x[idx][5]*t**5)
        supervisor.step(timestep)
        iterations += 1
    for i, joint in enumerate(joints):
        joint.setPosition(finger_joint_limits[i][close])
    timef = supervisor.getTime()
    print('Iterações totais: ', iterations)
    print(timef-time0)

In [52]:
actuate_gripper(0)

Iterações totais:  63
2.016


In [51]:
# bottle.getField('translation').setSFVec3f([-1.467559345817077, 1.7836801495521285e-06, 0.9004484925267724])

In [150]:
bottle.getField('translation').getSFVec3f()

[-1.467559345817077, 1.7836801495521285e-06, 0.9004484925267724]

In [149]:
supervisor.step(timestep)

0

In [50]:
move_to_config([0, 0, 0, 0, 0, 0])

Iterações totais:  157
5.024
0.1360267852177979
0.03290354636429906


In [ ]:
move_to_config([.5*pi, .05*pi, .45*pi, 0, -.5*pi, .5*pi])

In [29]:
move_to_config([-.5*pi, -.05*pi, -.45*pi, 0, .5*pi, .5*pi])

Iterações totais:  157
5.024
0.061558482575319484
0.02059982695085481


In [ ]:
move_to_config([pi/2, 0, 0, 0, 0, 0])

In [ ]:
move_to_config([.1*pi, .18*pi, .57*pi, 0.25*pi, 0.2, pi/2])

In [31]:
move_to_config([-.3*pi, -.04*pi, -.8*pi, -0.17*pi, -0.5, 0])

Iterações totais:  157
5.023999999999999
0.0560465910454353
0.016566226537031412


In [ ]:
move_to_config([0.2, .04*pi, .8*pi, 0.17*pi, 0.3, pi/2])